# Part 1: Loading, Profiling, & Cleaning
Group: The Tree Friends

## Task 0?: Environment Setup (Optional)

Here, we are installing the Python packages and downloading the datasets required to ensure we have the necessary tools for our analysis.

### Installing the Packages

In the following cell, we're installing the necessary Python packages for this part of our analysis.

In [1]:
%%capture

%pip install requests
%pip install pandas
%pip install geopandas

### Downloading the Data

![Image](https://magical-milk-bucket.nyc3.digitaloceanspaces.com/DataRetrieval_Screenshot.png)
*The portion highlighted in yellow indicates the button for accessing the data. It presents options to download the appropriate files.*

By exporting the data directly, we can access the comprehensive dataset from the City of Chicago, which consists of 770K rows and 49 columns, rather than the limited subset provided via the SODA API.

In [2]:
crashes_csv = 'Traffic_Crashes_-_Crashes.csv'
crashes_geojson = 'Traffic Crashes - Crashes.geojson'

The aforementioned filenames correspond to the files we'll be downloading in the next step, which will be integral to our analysis moving forward. We'll use a CSV for Pandas, as it's often more compact and efficient than JSON for large datasets.

The following function checks if the file exists locally. If not, it streams the download in chunks, ensuring efficiency, especially for larger files.

Please note, the duration may vary based on the speed of your internet connection.

In [3]:
import os
import requests

def download_data(url: str, fn: str = None) -> None:
    '''
    Downloads the file from specified link if it doesn't exist already.
    '''

    if fn is None:
        fn = url.split("/")[-1]

    if os.path.exists(fn):
        return

    with requests.get(url, stream=True) as r:
        r.raise_for_status() # Throw an error if an issue arises
        with open(fn, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

In [4]:
# base_url = 'https://data.cityofchicago.org/api'

# download_data(f'{base_url}/views/85ca-t3if/rows.json?accessType=DOWNLOAD', crashes_csv)
# download_data(f'{base_url}/geospatial/85ca-t3if?accessType=DOWNLOAD&method=export&format=GeoJSON', crashes_geojson)

When downloaded from the aforementioned endpoints, we can obtain the most recent versions of the dataset. Nonetheless, given intermittent access issues in Google Colab, we've chosen to store these files on the cloud. This not only provides a static version of the files but also serves as a reliable backup.

In [5]:
base_url = 'https://magical-milk-bucket.nyc3.digitaloceanspaces.com'

download_data(f'{base_url}/{crashes_csv}')
download_data(f'{base_url}/{crashes_geojson}')

## Task 1 - Data Importation & Transformation

Now since we have the dataset, we'll load it, get an initial understanding of its attributes, and perform the necessary transformations.

In [6]:
import pandas as pd

df = pd.read_csv(crashes_csv)

df.head().T # Transpose the table for a better view of the columns

,0,1,2,3,4
CRASH_RECORD_ID,000ececd00a4474d1abebf13221e4a97efd79a0322f117...,00e20a26e53d0c6806fdb89244115fc27bdb3b368e1d03...,018bf30065a8947fb73e947c0bd197536d746d60b49eb3...,01ddd522eb074a6cbec117c10de0cae2cb78d2a6dd94b2...,01eb75485f6215d7b60bc219666c21564ac2ce46c578fb...
RD_NO,JG360767,JG434999,JG412388,JG412232,JG412428
CRASH_DATE_EST_I,NaN,NaN,NaN,NaN,NaN
CRASH_DATE,07/29/2023 11:40:00 AM,09/22/2023 06:00:00 PM,09/05/2023 03:54:00 PM,09/05/2023 01:33:00 PM,09/05/2023 02:59:00 PM
POSTED_SPEED_LIMIT,30,30,30,35,30
TRAFFIC_CONTROL_DEVICE,NO CONTROLS,TRAFFIC SIGNAL,UNKNOWN,NO CONTROLS,NO CONTROLS
DEVICE_CONDITION,NO CONTROLS,FUNCTIONING PROPERLY,UNKNOWN,NO CONTROLS,NO CONTROLS
WEATHER_CONDITION,CLEAR,CLEAR,UNKNOWN,CLEAR,CLEAR
LIGHTING_CONDITION,DAYLIGHT,DAYLIGHT,UNKNOWN,DAYLIGHT,DAYLIGHT
FIRST_CRASH_TYPE,TURNING,REAR END,REAR END,SIDESWIPE SAME DIRECTION,REAR END


By performing `head()`, we can get a generalized idea of how our dataset looks. Similarly, we'll perform the same for the `GeoDataFrame`.

In [7]:
import geopandas as gpd

gdf = gpd.read_file(crashes_geojson)

gdf.head().T

,0,1,2,3,4
work_zone_type,NaN,NaN,NaN,NaN,NaN
injuries_fatal,0,0,0,0,0
workers_present_i,NaN,NaN,NaN,NaN,NaN
injuries_non_incapacitating,0,1,1,0,1
crash_record_id,198f8e0a5a47a1ccf052708875c54d14ae50aedc5c1427...,a339c809696e7787e44b9a9b1711acb4db7360154e4f57...,13edf825facf4eabbc8692711fd49ba492cdb18f1da5fe...,8ad1109c818e291a195b914bfb9fce177ec8e0cd03c8ab...,9aed1ff419292364a4520a876f88f587b0ef8c72a3c22b...
injuries_incapacitating,0,0,0,0,0
injuries_no_indication,3,1,0,1,4
latitude,41.953858398,41.770569861,41.960065664,41.659076075,42.011752805
lighting_condition,"DARKNESS, LIGHTED ROAD","DARKNESS, LIGHTED ROAD","DARKNESS, LIGHTED ROAD",DARKNESS,DARKNESS
street_no,2825,733,4251,2629,3199


### Profiling the Data

From an initial glance, we can see that there are slight differences between the columns in each dataframe. We'll run some code to highlight them.

In [8]:
# Match the columns for consistency
df.columns = df.columns.str.lower()

# Running it twice as the difference is what isn't in the other
diff1 = df.columns.difference(gdf.columns).to_list()
diff2 = gdf.columns.difference(df.columns).to_list()

diff1, diff2

(['location', 'not_right_of_way_i'], ['geometry', 'private_property_i'])

They containg two differentiating column names; however, `LOCATION` and `geometry` are representative of the same coordinates as seen above.

Next, to get a general overview of each dataset, we'll:

- Run `info()` to get details on the columns, memory usage, and more.
- Identify the percentage of missing values for each attribute.
- Calculate basic statistics for numerical columns with `describe()`.
- Identify the unique categories for categorical columns.

In [9]:
from IPython.display import HTML
from io import StringIO

# We're using StringIO to store the information from both dataframes
# into memory so we're able to display them side-by-side later on.

buf1 = StringIO()
df.info(buf=buf1)
info_df = buf1.getvalue()

buf2 = StringIO()
gdf.info(buf=buf2)
info_gdf = buf2.getvalue()

display(HTML(f"""
<div style="float: left; width: 50%;"><pre>{info_df}</pre></div>
<div style="float: right; width: 50%;"><pre>{info_gdf}</pre></div>
"""))

We note that both dataframes have an identical number of rows and columns as indicated, and they consume the same memory. Beyond these similarities, there are disparities in the data types used.

In [10]:
# Calculate the percentage of entries missing
df_missing_percentages = df.isna().mean() * 100
gdf_missing_percentages = gdf.isna().mean() * 100

# Filter out the features that aren't missing anything
dfm = df_missing_percentages[df_missing_percentages > 0].sort_values()
gdfm = gdf_missing_percentages[gdf_missing_percentages > 0].sort_values()

display(HTML(f"""
<div style="float: left; width: 50%;"><pre>{dfm}</pre></div>
<div style="float: right; width: 50%;"><pre>{gdfm}</pre></div>
"""))

Several columns ending in _I appear to have significant missing data, which we'll address in the next step.

We'll also be using the `describe()` function only on the dataframe, as GeoPandas no longer supports it. However, this shouldn't be a point of concern, considering we are working with the same dataset.

In [11]:
df.describe()

,posted_speed_limit,lane_cnt,street_no,beat_of_occurrence,num_units,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude
count,768535.000000,1.990050e+05,768535.000000,768530.000000,768535.000000,766860.000000,766860.000000,766860.000000,766860.000000,766860.000000,766860.000000,766860.0,768535.000000,768535.000000,768535.000000,763426.000000,763426.000000
mean,28.391139,1.333038e+01,3685.688306,1242.201924,2.034717,0.188225,0.001193,0.020147,0.106460,0.060424,2.005057,0.0,13.209112,4.123882,6.649006,41.854862,-87.673646
std,6.200848,2.961631e+03,2889.338175,705.390224,0.452438,0.563656,0.037457,0.166358,0.420662,0.316222,1.159794,0.0,5.563800,1.980332,3.365313,0.329216,0.668228
min,0.000000,0.000000e+00,0.000000,111.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.936193
25%,30.000000,2.000000e+00,1244.000000,713.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,9.000000,2.000000,4.000000,41.782297,-87.721629
50%,30.000000,2.000000e+00,3200.000000,1211.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,14.000000,4.000000,7.000000,41.874654,-87.673982
75%,30.000000,4.000000e+00,5600.000000,1822.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,17.000000,6.000000,9.000000,41.924259,-87.633227
max,99.000000,1.191625e+06,451100.000000,6100.000000,18.000000,21.000000,4.000000,10.000000,21.000000,15.000000,61.000000,0.0,23.000000,7.000000,12.000000,42.022780,0.000000


In [12]:
import json

# Filter potential 'categorical' columns with less than 100 unique values
filtered_columns = df.columns[df.apply(lambda col: col.dtype == 'object' and len(col.unique()) < 100)].tolist()

# Store the unique values for each column in a dictionary
unique_values_by_column = {col: df[col].sort_values().unique().tolist() for col in filtered_columns}

# Save the dictionary to an output file
with open('df_unique_values_by_column.json', 'w') as f:
    json.dump(unique_values_by_column, f, indent=4)

- The unique categorical values have been stored in a file to for visibility purposes.
- A threshold of 100 unique values; columns exceeding this count are likely not categorical in nature, for example, street names.

Upon a brief examination,
- Many values have missing entries (`NaN`).
- The columns `PRIM_CONTRIBUTORY_CAUSE` and `SEC_CONTRIBUTORY_CAUSE` have the same values.
- Columns ending in `_I` appear to be boolean in nature.
- The remaining columns display varied values with little to no similarity.

We'll check the latitudinal and longitudinal values for any points outside the limits of Chicago. Based on the missing values check, these values correspond to the location/geometry column.

In [13]:
long_min = -87.9401
lat_min = 41.6445
long_max = -87.5241
lat_max = 42.0230

# We're updating the datatypes in Part 2 but we'll do it here just for the comparisons
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')
gdf[['latitude', 'longitude']] = gdf[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')

# Get any points that are outside of the boundaries of the city.
out_of_bounds_df = df[(df['latitude'] < lat_min) | (df['latitude'] > lat_max) | (df['longitude'] < long_min) | (df['longitude'] > long_max)]
out_of_bounds_gdf = gdf[(gdf['latitude'] < lat_min) | (gdf['latitude'] > lat_max) | (gdf['longitude'] < long_min) | (gdf['longitude'] > long_max)]

# Count the number of points outside
out_of_bounds_df.shape[0], out_of_bounds_gdf.shape[0]

(44, 44)

### Cleaning the Data

Having identified underlying issues like missing values, we'll address them to guarantee consistent and reliable analyses. For efficiency, we'll make adjustments directly to the dataframe (using `inplace` if applicable). We won't need the original, and our source remains unaltered on the cloud.

We will start by removing any points outside the boundaries of the city.

In [14]:
# We'll apply them separately in case they are in different indices.
df.drop(index=out_of_bounds_df.index, inplace=True)
gdf.drop(index=out_of_bounds_gdf.index, inplace=True)

df.shape, gdf.shape

((768491, 49), (768491, 49))

Next, we'll omit the columns `CRASH_REPORT_ID` and `RD_NO` from our analysis. The former is relevant only when interfacing with supplementary datasets, and the latter merely represents a report identifier.

In [15]:
df.drop(columns=['crash_record_id', 'rd_no'], inplace=True)
gdf.drop(columns=['crash_record_id', 'rd_no'], inplace=True)

After evaluating the column types and their respective values above, we have determine:

- For the majority of columns, particularly those that detail specific incident attributes, we're choosing not to alter values to maintain the integrity and accuracy of our data.

- Columns like LANE_CNT and BEAT_OF_OCCURRENCE present a unique challenge. While there's a theoretical possibility of programmatic modification, the nature of the data combined with the dataset's large size makes such an endeavor both challenging and potentially infeasible.

- On the other hand, there are columns, such as WORK_ZONE_TYPE, where we can make modifications. However, we're deciding against it as we plan to discard this shortly.

In essence, given the reasons stated above, we won't be filling any missing values. However, we will be removing columns where more than 3% of the dataset is missing. It's evident that the columns vital for our domain-specific questions don't exceed this threshold. In other words, removing the rows won't have any negative consequences.

In [16]:
columns_to_drop_df = df_missing_percentages[df_missing_percentages > 3].index
columns_to_drop_gdf = gdf_missing_percentages[gdf_missing_percentages > 3].index

df.drop(columns=columns_to_drop_df, inplace=True)
gdf.drop(columns=columns_to_drop_gdf, inplace=True)

We are keeping the remaining columns in case we may want to expand our analysis in the future.

With the modifications in place, we can remove the rows with `NaN` values as they aren't beneficial. By using `shape`, we can see how much of the dataset has changed.

In [17]:
df.dropna(inplace=True)
gdf.dropna(inplace=True)

df.shape, gdf.shape

((739675, 36), (739675, 36))

Furthermore, we'll modify the categorical values for the contributory causes to shorten them for readability.

In [18]:
labels = {
    "ANIMAL": "Animal",
    "BICYCLE ADVANCING LEGALLY ON RED LIGHT": "Bicycle on Red",
    "CELL PHONE USE OTHER THAN TEXTING": "Cell Phone Use",
    "DISREGARDING OTHER TRAFFIC SIGNS": "Ignored Signs",
    "DISREGARDING ROAD MARKINGS": "Ignored Markings",
    "DISREGARDING STOP SIGN": "Ignored Stop",
    "DISREGARDING TRAFFIC SIGNALS": "Ignored Signals",
    "DISREGARDING YIELD SIGN": "Ignored Yield",
    "DISTRACTION - FROM INSIDE VEHICLE": "Inside Distraction",
    "DISTRACTION - FROM OUTSIDE VEHICLE": "Outside Distraction",
    "DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)": "Electronics Distraction",
    "DRIVING ON WRONG SIDE/WRONG WAY": "Wrong Side",
    "DRIVING SKILLS/KNOWLEDGE/EXPERIENCE": "Poor Skills",
    "EQUIPMENT - VEHICLE CONDITION": "Vehicle Issue",
    "EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST": "Evasive Action",
    "EXCEEDING AUTHORIZED SPEED LIMIT": "Over Limit",
    "EXCEEDING SAFE SPEED FOR CONDITIONS": "Unsafe Speed",
    "FAILING TO REDUCE SPEED TO AVOID CRASH": "No Speed Reduction",
    "FAILING TO YIELD RIGHT-OF-WAY": "Failed to Yield",
    "FOLLOWING TOO CLOSELY": "Tailgating",
    "HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)": "Drinking",
    "IMPROPER BACKING": "Bad Backing",
    "IMPROPER LANE USAGE": "Wrong Lane",
    "IMPROPER OVERTAKING/PASSING": "Unsafe Overtake",
    "IMPROPER TURNING/NO SIGNAL": "No Turn Signal",
    "MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT": "Motorcycle on Red",
    "NOT APPLICABLE": "N/A",
    "OBSTRUCTED CROSSWALKS": "Obstructed Walk",
    "OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER": "Reckless Driving",
    "PASSING STOPPED SCHOOL BUS": "Passed School Bus",
    "PHYSICAL CONDITION OF DRIVER": "Driver Condition",
    "RELATED TO BUS STOP": "Bus Stop Related",
    "ROAD CONSTRUCTION/MAINTENANCE": "Road Construction",
    "ROAD ENGINEERING/SURFACE/MARKING DEFECTS": "Road Defects",
    "TEXTING": "Texting",
    "TURNING RIGHT ON RED": "Right on Red",
    "UNABLE TO DETERMINE": "Unknown",
    "UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)": "DUI",
    "VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)": "Vision Blocked",
    "WEATHER": "Weather"
}

df['prim_contributory_cause'].replace(labels, inplace=True)
df['sec_contributory_cause'].replace(labels, inplace=True)

With the modifications in place, we'll export the results to files for use for Part 2. Additionally, a backup of these changes will be saved in the cloud for convenient access in Colab.

- The `GeoDataFrame` will be saved to the binary Feather format to enhance efficiency and speed up subsequent data loading.

In [19]:
# Use the same filename across both files
shared_fn = 'Traffic Crashes (Modified)'

df.to_csv(shared_fn + '.csv')
gdf.to_feather(shared_fn + '.feather')